# Optimize the Python propensity score matcher

Last updated: 2019-10-22

In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict
from sortedcontainers import SortedList

In [2]:
%load_ext line_profiler

## Read data

In [3]:
data = pd.read_csv("../data/scores.tsv", sep='\t', names=["MMI_ID", "pred_prob", "used_drug"])

In [4]:
data.shape

(1000000, 3)

In [5]:
data.head()

,MMI_ID,pred_prob,used_drug
0,0,0.746445,1
1,1,0.401794,1
2,2,0.207886,1
3,3,0.551139,1
4,4,0.522224,1


## Propensity matcher (original)

In [6]:
def match_pairs(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = people.assign(
        prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(int)
    )
    
    neg_ppl = scores.query("used_drug == 0")
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")
    for row in pos_ppl.itertuples():
        target = row.prop_score
        
        idx = neg_scores.bisect_left(target)
        
        if idx == len(neg_scores):
            idx -= 1
            
        if idx == -1:
            break
            
        closest_val = neg_scores[idx]
        
        if idx > 0:
            prev_val = neg_scores[idx - 1]
            
            if abs(target - prev_val) < abs(target - closest_val):
                closest_val = prev_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row.MMI_ID, closest_val))
        
    neg_ppl_list = defaultdict(list)
    for row in neg_ppl.itertuples():
        neg_ppl_list[row.prop_score].append(row.MMI_ID)
        
    matched_pairs = []
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs.append((pos_person, neg_person))
        
    final = pd.DataFrame(
        matched_pairs, columns=["MMI_ID", "matched_person"]
    )
    
    return final

In [7]:
%%timeit

res = match_pairs(data)

5.58 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%lprun -T ../pipeline/profiles/py_orig -f match_pairs match_pairs(data)


*** Profile printout saved to text file '../pipeline/profiles/py_orig'. 


## Test different optimizations

- Reduce memory usage by specifying integer size

In [9]:
# change prop score to unsigned 32bit integer

def match_pairs_dev2(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = people.assign(
        prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(np.uint32)
    )
    
    neg_ppl = scores.query("used_drug == 0")
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")
    for row in pos_ppl.itertuples():
        target = row.prop_score
        
        idx = neg_scores.bisect_left(target)
        
        if idx == len(neg_scores):
            idx -= 1
            
        if idx == -1:
            break
            
        closest_val = neg_scores[idx]
        
        if idx > 0:
            prev_val = neg_scores[idx - 1]
            
            if abs(target - prev_val) < abs(target - closest_val):
                closest_val = prev_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row.MMI_ID, closest_val))
        
    neg_ppl_list = defaultdict(list)
    for row in neg_ppl.itertuples():
        neg_ppl_list[row.prop_score].append(row.MMI_ID)
        
    matched_pairs = []
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs.append((pos_person, neg_person))
        
    final = pd.DataFrame(
        matched_pairs, columns=["MMI_ID", "matched_person"]
    )
    
    return final

In [10]:
%%timeit

res = match_pairs_dev2(data)

5.56 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%lprun -T ../pipeline/profiles/py_dev2 -f match_pairs_dev2 match_pairs_dev2(data)


*** Profile printout saved to text file '../pipeline/profiles/py_dev2'. 


Using 32 bit integers seems to have made no difference.

## Optimization 3

- Rewrite inner loop to eliminate an expensive lookup

In [12]:
# change prop score to unsigned 32bit integer

# rewrite inner loop logic to eliminate an expensive index call when idx == n

def match_pairs_dev3(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = people.assign(
        prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(np.uint32)
    )
    
    neg_ppl = scores.query("used_drug == 0")
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")
    
    
    for row in pos_ppl.itertuples():
        
        
        if not neg_scores:
            break
        
        
        
        target = row.prop_score
        
        
        closest_val = None        
        
                
        idx = neg_scores.bisect_left(target)
        
        
        if idx == len(neg_scores):
            closest_val = neg_scores[-1]
        elif idx == 0:
            closest_val = neg_scores[0]

            
        if closest_val is None:
            cur_val = neg_scores[idx]
            prev_val = neg_scores[idx - 1]
            
            if (target - prev_val) < (cur_val - target):
                closest_val = prev_val
            else:
                closest_val = cur_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row.MMI_ID, closest_val))
        
    neg_ppl_list = defaultdict(list)
    for row in neg_ppl.itertuples():
        neg_ppl_list[row.prop_score].append(row.MMI_ID)
        
    matched_pairs = []
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs.append((pos_person, neg_person))
        
    final = pd.DataFrame(
        matched_pairs, columns=["MMI_ID", "matched_person"]
    )
    
    return final

In [13]:
%%timeit

res = match_pairs_dev3(data)

5.35 s ± 24.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Rewriting the logic reduced runtime by a very small 0.2 s.**

In [14]:
%lprun -T ../pipeline/profiles/py_dev3 -f match_pairs_dev3 match_pairs_dev3(data)


*** Profile printout saved to text file '../pipeline/profiles/py_dev3'. 


## Optimization 4

- Reduce pandas dataframe prior to iteration

In [15]:
# change prop score to unsigned 32bit integer

# rewrite inner loop logic to eliminate an expensive index call when idx == n

# reduce data size a little

def match_pairs_dev4(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = people.assign(
        prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(np.uint32)
    )
    
    neg_ppl = scores.query("used_drug == 0")[["MMI_ID", "prop_score"]]
    
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")[["MMI_ID", "prop_score"]]
    
    
    for row in pos_ppl.itertuples():
        
        if not neg_scores:
            break
        
        target = row.prop_score
        
        closest_val = None        
        
        idx = neg_scores.bisect_left(target)
        
        if idx == len(neg_scores):
            closest_val = neg_scores[-1]
        elif idx == 0:
            closest_val = neg_scores[0]

            
        if closest_val is None:
            cur_val = neg_scores[idx]
            prev_val = neg_scores[idx - 1]
            
            if (target - prev_val) < (cur_val - target):
                closest_val = prev_val
            else:
                closest_val = cur_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row.MMI_ID, closest_val))
        
    neg_ppl_list = defaultdict(list)
    for row in neg_ppl.itertuples():
        neg_ppl_list[row.prop_score].append(row.MMI_ID)
        
    matched_pairs = []
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs.append((pos_person, neg_person))
        
    final = pd.DataFrame(
        matched_pairs, columns=["MMI_ID", "matched_person"]
    )
    
    return final

In [16]:
%%timeit

res = match_pairs_dev4(data)

5.26 s ± 83.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Again it seems to have made a small 0.1 s difference.**

In [17]:
%lprun -T ../pipeline/profiles/py_dev4 -f match_pairs_dev4 match_pairs_dev4(data)


*** Profile printout saved to text file '../pipeline/profiles/py_dev4'. 


## Optimization 5

- Return result as a list, not as a dataframe

In [18]:
# change prop score to unsigned 32bit integer

# rewrite inner loop logic to eliminate an expensive index call when idx == n

# remove output dataframe

def match_pairs_dev5(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = people.assign(
        prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(np.uint32)
    )
    
    
    neg_ppl = scores.query("used_drug == 0")[["MMI_ID", "prop_score"]]
    
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")[["MMI_ID", "prop_score"]]
    
    
    for row in pos_ppl.itertuples():
        
        if not neg_scores:
            break
        
        target = row.prop_score
        
        closest_val = None        
        
        idx = neg_scores.bisect_left(target)
        
        if idx == len(neg_scores):
            closest_val = neg_scores[-1]
        elif idx == 0:
            closest_val = neg_scores[0]

            
        if closest_val is None:
            cur_val = neg_scores[idx]
            prev_val = neg_scores[idx - 1]
            
            if (target - prev_val) < (cur_val - target):
                closest_val = prev_val
            else:
                closest_val = cur_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row.MMI_ID, closest_val))

        
    neg_ppl_list = defaultdict(list)
    for row in neg_ppl.itertuples():
        neg_ppl_list[row.prop_score].append(row.MMI_ID)
        
        
        
    matched_pairs = defaultdict(list)
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs["MMI_ID"].append(pos_person)
        matched_pairs["matched_person"].append(neg_person)
        
        
    return matched_pairs

In [19]:
%%timeit

res = match_pairs_dev5(data)

5.03 s ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Eliminating the use of the return dataframe provides another small 0.2 s speed up.

## Optimization 6

- Convert dataframe to numpy array prior to looping

In [20]:
# change prop score to unsigned 32bit integer

# rewrite inner loop logic to eliminate an expensive index call when idx == n

# remove output dataframe

# changed to numpy for looping


def match_pairs_dev6(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = (people
        .assign(
            prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(np.uint32)
        )
        .drop("pred_prob", axis=1)
    )
    
    
    neg_ppl = scores.query("used_drug == 0")[["MMI_ID", "prop_score"]]
    
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")[["MMI_ID", "prop_score"]].to_numpy()
    
    
    for row in pos_ppl:
        
        if not neg_scores:
            break
        
        target = row[1] # prop_score
        
        
        closest_val = None        
        
                
        idx = neg_scores.bisect_left(target)
        
        if idx == len(neg_scores):
            closest_val = neg_scores[-1]
        elif idx == 0:
            closest_val = neg_scores[0]

            
        if closest_val is None:
            cur_val = neg_scores[idx]
            prev_val = neg_scores[idx - 1]
            
            if (target - prev_val) < (cur_val - target):
                closest_val = prev_val
            else:
                closest_val = cur_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row[0], closest_val))

        
    neg_ppl_list = defaultdict(list)
    
    neg_ppl = neg_ppl.to_numpy()
    
    for row in neg_ppl:
        neg_ppl_list[row[1]].append(row[0])
        
        
        
    matched_pairs = defaultdict(list)
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs["MMI_ID"].append(pos_person)
        matched_pairs["matched_person"].append(neg_person)
        
        
    return matched_pairs

In [21]:
%%timeit

res = match_pairs_dev6(data)

5.87 s ± 79.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

In [22]:
# change prop score to unsigned 32bit integer

# rewrite inner loop logic to eliminate an expensive index call when idx == n

# remove output dataframe

# changed to numpy for looping


def match_pairs_dev7(people, caliper=0.05):
    
    PRECISION = 1000000
    MAX_DIST = int(caliper * PRECISION)
    
    scores = people.assign(
        prop_score = lambda df: np.floor(df["pred_prob"].mul(PRECISION)).astype(np.uint32)
    )

              
    
    neg_ppl = scores.query("used_drug == 0")[["MMI_ID", "prop_score"]]
    
    
    neg_scores = SortedList(neg_ppl["prop_score"])
    
    matched_scores = []
    pos_ppl = scores.query("used_drug == 1")[["MMI_ID", "prop_score"]].to_numpy()
    
    
    for row in pos_ppl:
        
        
        if not neg_scores:
            break
        
        
        target = row[1] # prop_score
        
        
        closest_val = None        
        
                
        idx = neg_scores.bisect_left(target)
        
        
        if idx == len(neg_scores):
            closest_val = neg_scores[-1]
        elif idx == 0:
            closest_val = neg_scores[0]

            
        if closest_val is None:
            cur_val = neg_scores[idx]
            prev_val = neg_scores[idx - 1]
            
            if (target - prev_val) < (cur_val - target):
                closest_val = prev_val
            else:
                closest_val = cur_val
                
        if abs(target - closest_val) > MAX_DIST:
            continue
            
        neg_scores.remove(closest_val)
        
        matched_scores.append((row[0], closest_val))

        
    neg_ppl_list = defaultdict(list)
    
    neg_ppl = neg_ppl.to_numpy()
    
    for row in neg_ppl:
        neg_ppl_list[row[1]].append(row[0])
        
        
        
    matched_pairs = defaultdict(list)
    for pos_person, pscore in matched_scores:
        neg_person = neg_ppl_list[pscore].pop()
        
        matched_pairs["MMI_ID"].append(pos_person)
        matched_pairs["matched_person"].append(neg_person)
        
        
    return matched_pairs

In [23]:
%%timeit

res = match_pairs_dev7(data)

5.82 s ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Not sure what happened, previously using numpy would speed up the looping, but now it's making things worse.

Previously using numpy prior to looping would reduce runtime by another 0.5 s.

# Conclusion

At this point it is clear that it will not really be possible to speed up the function in Python without some serious work. Python is just a slow language, and instead of optimizing Python, we should rewrite and optimize in the much faster C++, and call the fast C++ function from Python.